<a href="https://colab.research.google.com/github/AmirPupko/pandas-to-sql/blob/main/pandas_to_sql_colab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-to-sql -U

Requirement already up-to-date: pandas-to-sql in /usr/local/lib/python3.6/dist-packages (0.0.546)


In [2]:
from copy import copy
import sqlite3
import pandas as pd
import pandas_to_sql
from pandas_to_sql import conventions

In [3]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
table_name = 'iris'
sql_connection = sqlite3.connect('./iris.db') #create db
iris.to_sql(table_name, sql_connection, if_exists='replace', index=False)
iris[:2]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [4]:
df = pandas_to_sql.wrap_df(iris, table_name)
pd_wrapped = pandas_to_sql.wrap_pd(pd)

In [5]:
df.get_sql_string()

'SELECT (sepal_length) AS sepal_length, (sepal_width) AS sepal_width, (petal_length) AS petal_length, (petal_width) AS petal_width, (species) AS species FROM iris'

In [6]:
species_petal_length_stats_df = df.groupby('species').agg({'petal_length':['mean','sum','count']})
species_petal_length_stats_df = conventions.flatten_grouped_dataframe(species_petal_length_stats_df)

species_petal_length_stats_df.get_sql_string()


'SELECT (avg(petal_length)) AS petal_length_mean, (sum(petal_length)) AS petal_length_sum, (count(petal_length)) AS petal_length_count, (species) AS species FROM iris GROUP BY species'

In [7]:
df[(df.petal_length>1.4) & (df.petal_width<.2)].get_sql_string()

'SELECT (sepal_length) AS sepal_length, (sepal_width) AS sepal_width, (petal_length) AS petal_length, (petal_width) AS petal_width, (species) AS species FROM iris WHERE (((petal_length > 1.4) AND (petal_width < 0.2))) '

In [8]:
df_ = copy(df)
df_['sepal_width_rounded'] = df_.sepal_width.round()
df_1 = df_[df_.species=='setosa'].reset_index(drop=True)
df_2 = df_[df_.species=='versicolor'].reset_index(drop=True)
pd_wrapped.concat([df_1, df_2]).get_sql_string()

"SELECT (sepal_length) AS sepal_length, (sepal_width) AS sepal_width, (petal_length) AS petal_length, (petal_width) AS petal_width, (species) AS species, ((CASE WHEN ((ABS(sepal_width) - ROUND(ABS(sepal_width)-0.5))==.5) AND ((CAST(sepal_width AS INT))%2 == 0) THEN (CASE WHEN sepal_width>0 THEN ROUND(sepal_width-0.001) ELSE ROUND(sepal_width+0.001) END) ELSE (ROUND(sepal_width)) END)) AS sepal_width_rounded FROM iris WHERE ((species = 'setosa'))  UNION ALL SELECT (sepal_length) AS sepal_length, (sepal_width) AS sepal_width, (petal_length) AS petal_length, (petal_width) AS petal_width, (species) AS species, ((CASE WHEN ((ABS(sepal_width) - ROUND(ABS(sepal_width)-0.5))==.5) AND ((CAST(sepal_width AS INT))%2 == 0) THEN (CASE WHEN sepal_width>0 THEN ROUND(sepal_width-0.001) ELSE ROUND(sepal_width+0.001) END) ELSE (ROUND(sepal_width)) END)) AS sepal_width_rounded FROM iris WHERE ((species = 'versicolor')) "

In [9]:
df_ = copy(df)
df_['sepal_width_rounded'] = df_.sepal_width.round()
df_1 = df_[df_.species=='setosa'].reset_index(drop=True)
df_2 = df_[df_.species=='versicolor'].reset_index(drop=True)

some_df = pd_wrapped.concat([df_1, df_2])

sql_string = some_df.get_sql_string()

df_from_sql_database = pd.read_sql_query(sql_string, sql_connection)
df_pandas = some_df.df_pandas

from pandas_to_sql.testing.utils.asserters import assert_dataframes_equals
assert_dataframes_equals(df_pandas, df_from_sql_database)